In [18]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import ParameterGrid
import xgboost as xgb

from sklearn.model_selection import cross_val_score


In [7]:
X = pd.read_pickle('dataframes/X_filled_random_undersampled.pkl')
y = pd.read_pickle('dataframes/y_filled_random_undersampled.pkl')

In [24]:
y.value_counts()

0.0    24825
1.0    24825
Name: TARGET, dtype: int64

In [40]:
def my_metrics(y_preds, y, yup):
    scores = list()
    for i in range(0, len(y_preds)):
        if ( y_preds[i] == 0 ) & ( y[i] == 0 ):
            scores.append(0)
        elif ( y_preds[i] == 1 ) & ( y[i] == 1 ):
            scores.append(0.1)
        elif ( y_preds[i] == 1 ) & ( y[i] == 0 ):
            scores.append(-0.1)
        elif ( y_preds[i] == 0 ) & ( y[i] == 1 ):
            scores.append(-1)
    return np.array(scores)

In [41]:
def XGBClassifier_train(X, y):
    # Inspired by https://towardsdatascience.com/beginners-guide-to-xgboost-for-classification-problems-50f75aac5390
    # Preset the hyper-params.
    param_grid = ParameterGrid(
        {
            "gamma": [0.5],
            "reg_alpha": [0, 1],
            "max_depth": [1],
            "learning_rate": [0.01],
        }
    )

    best_score = 0
    nb_comb = len(list(param_grid))

    XGB_dic = dict()

    #fb_score = make_scorer(fbeta_score, beta=3)

    i=0
    for params in param_grid:
        i+=1
        print(i, "/", nb_comb, sep="")
        # Declare the model
        xgb_cl = xgb.XGBClassifier(**params)

        score = np.mean(cross_val_score(xgb_cl, X, y, cv=5, scoring=my_metrics))
        XGB_dic[str(params)] = score

        print(score)
        if 1: #score > best_score:
            print(score, params)
            best_score = score
            best_params = params

    with open('XGB_dic.pkl', 'wb') as f:
        pickle.dump(XGB_dic, f)

    return {"mehtod": "XGB", "best_score": best_score, "params": best_params}

In [42]:
XGBClassifier_train(X, y)

1/2


/home/nhey/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/nhey/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/nhey/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 108, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/tmp/ipykernel_32233/3809800598.py", line 3, in my_metrics
    for i in range(0, len(y_preds)):
TypeError: object of type 'XGBClassifier' has no len()

  warnings.warn(


nan
nan {'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 1, 'reg_alpha': 0}
2/2


/home/nhey/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/nhey/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/nhey/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 108, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/tmp/ipykernel_32233/3809800598.py", line 3, in my_metrics
    for i in range(0, len(y_preds)):
TypeError: object of type 'XGBClassifier' has no len()

  warnings.warn(


nan
nan {'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 1, 'reg_alpha': 1}


{'mehtod': 'XGB',
 'best_score': nan,
 'params': {'gamma': 0.5,
  'learning_rate': 0.01,
  'max_depth': 1,
  'reg_alpha': 1}}